In [1]:
import numpy as np
import glob
import os, re
import cv2
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

In [ ]:
!pwd

In [2]:
dir = "/Users/tomi/Astronomy/MTA_CSFK/ML_OGLE/OGLE3/test/testdata_nocolor_withtxt/"

In [3]:
datafiles = []
datafiles = sorted(glob.glob(dir + "/*.txt"))
print("Number of samples: ", len(datafiles))

Number of samples:  5000


In [4]:
datasize = len(datafiles)
train_setsize = 0.8
test_setsize = 0.2
img_size = 128
batch = 32
epoch_number = 25

X = []
y = []

for i in datafiles:
    datafile = np.genfromtxt(i)
    label = os.path.basename(i).split("_")[0]
    X.append(datafile)
    y.append(label)

X=np.array(X)
y=np.array(y)

In [ ]:
#print("Datasize:", datasize)
#print("Train files:", len(train_set))
#print("Test files:", len(test_set))
#print("Control files:", len(control_set))
y

In [5]:
import sklearn.preprocessing
encoder = sklearn.preprocessing.OneHotEncoder(sparse=False)

encoder.fit(y.reshape(-1,1))
y_enc = encoder.transform(y.reshape(-1,1))

In [6]:
import sklearn.model_selection
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y_enc, 
                                                                            shuffle=True, 
                                                                            stratify=y, 
                                                                            train_size = train_setsize,
                                                                            test_size=test_setsize, 
                                                                            random_state=42)

# (n_sampl, x_dim, y_dim, channels) = X_train.shape


In [41]:
y_train.shape

(4000, 5)

In [42]:
# Test network version 0.1

# from keras import Sequential
# from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
# from keras.layers import Flatten,Dense,Dropout

from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Input

# x = Sequential()

# x.add( Flatten() )

# x.add( Dense(256, activation='relu'))
# x.add( Dense(256, activation='relu'))
# x.add( Dense(64, activation='relu'))

# x.add( Dense(5, activation='softmax'))

# # inputs = Input(shape=(X_train.shape[0],))
inputA = Input(shape=(1,))
# x = Flatten()(inputA)

x = Dense(256, activation='relu')(inputA)
x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)

x = Dense(5, activation='softmax')(x)

x = Model(inputs=inputA, outputs = x)


In [43]:
# from tensorflow.keras.optimizers import SGD

# opt = SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
x.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [44]:
x.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               512       
_________________________________________________________________
dense_23 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_24 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_25 (Dense)             (None, 5)                 325       
Total params: 83,077
Trainable params: 83,077
Non-trainable params: 0
_________________________________________________________________


In [45]:
x.fit(X_train, y_train, 
          validation_data=(X_test, y_test),
          batch_size=batch, 
          verbose=1, 
          epochs=epoch_number)

# x.fit(X_train, y_train)

Train on 4000 samples, validate on 1000 samples
Epoch 1/25
4000/4000 [==============================] - 1s 362us/sample - loss: 1.4806 - accuracy: 0.3435 - val_loss: 1.4056 - val_accuracy: 0.5180
Epoch 2/25
4000/4000 [==============================] - 0s 117us/sample - loss: 1.3700 - accuracy: 0.6390 - val_loss: 1.7336 - val_accuracy: 0.6260
Epoch 3/25
4000/4000 [==============================] - 0s 124us/sample - loss: 1.3386 - accuracy: 0.6308 - val_loss: 1.2261 - val_accuracy: 0.6820
Epoch 4/25
4000/4000 [==============================] - 1s 127us/sample - loss: 1.3164 - accuracy: 0.6370 - val_loss: 1.1941 - val_accuracy: 0.6320
Epoch 5/25
4000/4000 [==============================] - 1s 130us/sample - loss: 1.2173 - accuracy: 0.6505 - val_loss: 1.1600 - val_accuracy: 0.6630
Epoch 6/25
4000/4000 [==============================] - 0s 118us/sample - loss: 1.2395 - accuracy: 0.6410 - val_loss: 1.1838 - val_accuracy: 0.6460
Epoch 7/25
4000/4000 [==============================] - 0s 118us

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print ('Loss, accuracy:', test_loss, test_accuracy)


In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print ('Loss, accuracy:', test_loss, test_accuracy)

In [ ]:
import keras.utils
from keras.utils.vis_utils import plot_model
# import tensorflow.keras.utils.plot_model as modelplot
# tensorflow.keras.utils.
keras.utils.print_summary(model)
plot_model(model, to_file='model.png')


In [ ]:
import tensorflow.keras.utils as utils

# utils.plot_model(model, to_file='model.png', show_layer_names=True)
utils.plot_model(model, show_layer_names=True)

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)